# Food Classification

![](../img2/food-101.jpg)
### Food 101 dataset

-101 class

-1000 images per class

https://www.vision.ee.ethz.ch/datasets_extra/food-101/

#### code will be available starting from wednesday Dec 6th on
https://github.com/asemyanov

# Transfer Learning

<img src="../img2/topology.png" alt="Drawing" style="width: 600px;"/>



<img src="../img2/finetuning.png" alt="Drawing" style="width: 900px;"/>



<img src="../img2/lr2.png" alt="Drawing" style="width: 700px;"/>



In [1]:
import mxnet as mx
import numpy as np
from mxnet import ndarray as nd
from mxnet import gluon
from mxnet.gluon.model_zoo.vision import resnet152_v1
from mxnet.image import color_normalize
from mxnet import autograd

import time

/home/ubuntu/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/urllib3/contrib/pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL


In [2]:
from skimage import io, transform

In [3]:
input_size = 224, 224
batch_size = 64
num_classes = 101
num_gpus = 1

In [4]:
batch_size *= max(1, num_gpus)
context = [mx.gpu(i) for i in range(num_gpus)] if num_gpus > 0 else [mx.cpu()]

In [5]:
def transformer(data, label):
    data = mx.image.imresize(data, 224, 224)
    data = mx.nd.transpose(data, (2,0,1))
    data = data.astype(np.float32) / 255
    data = color_normalize(
        data,
        mean=mx.nd.array([0.485, 0.456, 0.406]).reshape((3,1,1)),
        std=mx.nd.array([0.229, 0.224, 0.225]).reshape((3,1,1))
    )
    return data, label

In [8]:
train_data = gluon.data.vision.ImageRecordDataset('../exp_data/mydata_train.rec', transform=transformer)
val_data = gluon.data.vision.ImageRecordDataset('../exp_data/mydata_val.rec', transform=transformer)

In [9]:
class RecordSampler(gluon.data.Sampler):
    def __init__(self, dataset, shuffle=False):
        self._index = dataset._record.keys
        self._shuffle = shuffle
        
    def __len__(self):
        return len(self._index)
    
    def __iter__(self):
        if self._shuffle:
            self._index = np.random.permutation(self._index)
        for idx in self._index:
            yield idx

In [10]:
train_loader = gluon.data.DataLoader(train_data, batch_size, sampler=RecordSampler(train_data, True))
val_loader = gluon.data.DataLoader(val_data, batch_size, sampler=RecordSampler(val_data))

In [19]:
len (train_loader)

1500

In [20]:
for i, (d, l) in enumerate(train_loader):
    if i%100 == 0:
        print ('i=',i, 'shape=',d.shape)
    else:
        pass

i= 0 shape= (64, 3, 224, 224)
i= 100 shape= (64, 3, 224, 224)
i= 200 shape= (64, 3, 224, 224)
i= 300 shape= (64, 3, 224, 224)
i= 400 shape= (64, 3, 224, 224)
i= 500 shape= (64, 3, 224, 224)
i= 600 shape= (64, 3, 224, 224)
i= 700 shape= (64, 3, 224, 224)
i= 1000 shape= (64, 3, 224, 224)
i= 1100 shape= (64, 3, 224, 224)
i= 1200 shape= (64, 3, 224, 224)
i= 1300 shape= (64, 3, 224, 224)
i= 1400 shape= (64, 3, 224, 224)


In [14]:
d.shape

(64, 3, 224, 224)

#### Alternative dataset loader: Just your folder with subfolders with the names of your classes

In [22]:
root = '/home/ubuntu/food-101/images/'
train2 = gluon.data.vision.ImageFolderDataset (root, flag=1, transform=transformer)
mysampler = gluon.data.RandomSampler(train2.__len__()) 
mybatchsampler = gluon.data.BatchSampler(mysampler, batch_size=20, last_batch='discard')
#list(mybatchsampler)
print (train2.__getitem__(0)[0].shape)

train_data2 = gluon.data.DataLoader(train2, batch_sampler=mybatchsampler)
for d, l in train_data2:
    break
    
print (d.shape)

(3, 224, 224)
(20, 3, 224, 224)


### Define the model that you are using. 
#### using Gluon model Zoo: Resnet-152

In [23]:
vanilla_resnet = resnet152_v1(pretrained=True, ctx=context)
finetuned_network = resnet152_v1(ctx=context, classes=101)
finetuned_network.collect_params().initialize(ctx=context)
finetuned_network.features = vanilla_resnet.features

In [24]:
finetuned_network.hybridize()

In [26]:
def metric_str(names, accs):
    return ', '.join(['%s=%f'%(name, acc) for name, acc in zip(names, accs)])
metric = mx.metric.create(['acc'])

In [27]:
def evaluate(net, dataset, ctx):
    for batch in dataset:
        data = gluon.utils.split_and_load(batch[0], ctx_list=ctx, batch_axis=0)
        label = gluon.utils.split_and_load(batch[1], ctx_list=ctx, batch_axis=0)
        outputs = []
        for x in data:
            outputs.append(net(x))
        metric.update(label, outputs)
    out = metric.get()
    metric.reset()
    return out

In [29]:
def train(net, train_dataset, val_dataset, epochs, ctx, log_interval=10):
    if isinstance(ctx, mx.Context):
        ctx = [ctx]
    trainer = gluon.Trainer(net.clasifier.collect_params(), 'sgd', {'learning_rate': 1e-3, 'wd': 1e-7})
    loss = gluon.loss.SoftmaxCrossEntropyLoss(from_logits=True)

    best_f1 = 0
    for epoch in range(epochs):
        tic = time.time()
        btic = time.time()
        for i, batch in enumerate(train_dataset):
            data = gluon.utils.split_and_load(batch[0], ctx_list=ctx, batch_axis=0)
            label = gluon.utils.split_and_load(batch[1], ctx_list=ctx, batch_axis=0)
            outputs = []
            Ls = []
            with autograd.record():
                for x, y in zip(data, label):
                    z = net(x)
                    L = loss(z, y)
                    Ls.append(L)
                    outputs.append(z)
                for L in Ls:
                    L.backward()
            trainer.step(batch[0].shape[0])
            metric.update(label, outputs)
            if log_interval and not (i+1)%log_interval:
                names, accs = metric.get()
                print('[Epoch %d Batch %d] speed: %f samples/s, training: %s'%(
                      epoch, i, batch_size/(time.time()-btic), metric_str(names, accs)))
            
            btic = time.time()

        names, accs = metric.get()
        metric.reset()
        print('[Epoch %d] training: %s'%(epoch, metric_str(names, accs)))
        print('[Epoch %d] time cost: %f'%(epoch, time.time()-tic))
        val_names, val_accs = evaluate(net, val_dataset, ctx)
        print('[Epoch %d] validation: %s'%(epoch, metric_str(val_names, val_accs)))

        if val_accs[1] > best_f1:
            best_f1 = val_accs[1]
            logging.info('Best validation f1 found. Checkpointing...')
            net.save_params('deep-dog-%d.params'%(epoch))

## Start training

In [30]:
train(finetuned_network, train_data2, train_data2, 10, context)

[Epoch 0 Batch 9] speed: 243.192335 samples/s, training: accuracy=0.015000
[Epoch 0 Batch 19] speed: 262.834968 samples/s, training: accuracy=0.015000
[Epoch 0 Batch 29] speed: 261.724321 samples/s, training: accuracy=0.013333
[Epoch 0 Batch 39] speed: 244.586332 samples/s, training: accuracy=0.012500
[Epoch 0 Batch 49] speed: 275.396375 samples/s, training: accuracy=0.012000
[Epoch 0 Batch 59] speed: 270.824944 samples/s, training: accuracy=0.010833
[Epoch 0 Batch 69] speed: 240.441350 samples/s, training: accuracy=0.010714
[Epoch 0 Batch 79] speed: 268.061778 samples/s, training: accuracy=0.009375
[Epoch 0 Batch 89] speed: 269.087185 samples/s, training: accuracy=0.009444
[Epoch 0 Batch 99] speed: 239.399632 samples/s, training: accuracy=0.010000
[Epoch 0 Batch 109] speed: 258.419379 samples/s, training: accuracy=0.010455
[Epoch 0 Batch 119] speed: 264.997380 samples/s, training: accuracy=0.011250
[Epoch 0 Batch 129] speed: 245.864820 samples/s, training: accuracy=0.012692
[Epoch 0 B

KeyboardInterrupt: 

### Saving the model

In [ ]:
network = finetuned_network.save_params('finetuned_network.params', ctx=context)